In [30]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1, max_mem_size = '2g', ip = "127.0.0.1", port = 54321)


Checking whether there is an H2O instance running at http://127.0.0.1:54321 . connected.


H2O_cluster_uptime:,1 hour 7 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,"14 days, 23 hours and 15 minutes"
H2O_cluster_name:,H2O_from_python_frikk_gcnf0f
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.038 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [31]:
import pandas as pd
import os
import numpy as np

In [32]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)

folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)

In [33]:
#Get target
target = pd.read_csv("../data/stores_train.csv")
target = target["revenue"].values

target = np.log1p(target)

In [34]:
features_train["revenue"] = target

In [35]:
features_train.to_csv("h2o_automl_2_features_csv/features_train.csv", index=False)

features_test.to_csv("h2o_automl_2_features_csv/features_test.csv", index=False)

In [36]:
df_train = h2o.import_file("h2o_automl_2_features_csv/features_train.csv")
df_test = h2o.import_file("h2o_automl_2_features_csv/features_test.csv")

df_test.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:8577
Cols:19

,average_income_per_grunnkrets,numberOfBusstop,categorical_chain_name,categorical_mall_name,categorical_plaace,categorical_sales_channel_name,categorical_street,distance_store_busstop,distance_to_competitor,has_mall_or_not,number_adults_grunnkrets,children_number,couple_number,features,person_number,singles_number,person_concentration,area_grunnkrets,feature
type,real,int,enum,enum,enum,enum,enum,real,real,int,real,real,real,int,real,real,real,real,int
mins,0.0,0.0,,,,,,2.834648843907257,0.0,0.0,1.0,0.0,0.0,0.0,66.0,0.0,11.403680684347426,0.0034185853132582,0.0
mean,559440.6715634838,15.12277019937041,,,,,,363.42705370954155,1049.3819808456349,0.17407018771132096,297.22055355591414,129.97669228861892,170.24356412366285,0.6181648595079865,734.0515745102902,262.11037968731637,3408.4551972697577,0.7512256878394341,0.6044071353620147
maxs,799720.0,72.0,,,,,,40354.63748683241,128872.47008953274,1.0,2567.0,1789.0,1012.0,18.0,5949.0,2002.0,150733.47216561993,14.995158557421975,13.0
sigma,68128.77375037978,12.834269924280061,,,,,,1223.2054558757015,4092.9541128555234,0.37919193248133803,270.2875134203443,137.68181653627445,145.7888696438315,1.829245531857849,585.2326052568449,234.67556085195426,5688.329507465651,1.119024389594291,1.4938756118063603
zeros,27,377,,,,,,0,2144,7084,0,33,203,6880,0,21,0,0,6403
missing,0,0,6099,7084,0,0,1239,0,0,0,0,0,0,0,0,0,0,0,0
0,551600.0,21.0,,,1.1.1.0,Hamburger restaurants,STRÃ˜MSVEIEN,25.63006058192599,70.08576024451624,0.0,493.0,224.0,176.0,0.0,970.0,391.0,17009.352876611734,0.0570274487828266,0.0
1,531240.0,11.0,BURGER KING,,1.1.1.0,Hamburger restaurants,MYREN,111.73618940669482,177.7104392523965,0.0,297.22055355591414,129.9766922886189,0.0,0.0,734.0515745102902,3.0,4422.184269953098,0.1659929866554556,0.0
2,523300.0,11.0,BURGER KING,Stovner Senter,1.1.1.0,Hamburger restaurants,STOVNER SENTER,404.825119221575,0.0,1.0,1035.0,566.0,354.0,0.0,2348.0,1137.0,9922.759804281512,0.2366277171182634,0.0


In [37]:
# TRAIN

for col in features_train.columns:
    if col.startswith("categorical"):
        print(col)
        df_train[col] = df_train[col].asfactor()

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


In [38]:
# TEST

for col in features_test.columns:
    if col.startswith("categorical"):
        print(col)
        df_test[col] = df_test[col].asfactor()

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


In [39]:
y = "revenue"
x = df_train.columns
x.remove(y)

In [40]:
aml = H2OAutoML(sort_metric='rmse', max_runtime_secs=10*60, seed=1)
aml.train(x=x, y="revenue", training_frame = df_train)

AutoML progress: |
15:11:31.929: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.5440061,0.0096458,0.5354832,0.5557187,0.5326796,0.5487649,0.5473837
mean_residual_deviance,0.5167063,0.0147596,0.4994886,0.5306669,0.5022959,0.5226851,0.5283949
mse,0.5167063,0.0147596,0.4994886,0.5306669,0.5022959,0.5226851,0.5283949
null_deviance,2701.5957,106.74364,2664.5046,2664.135,2827.9321,2788.6108,2562.7964
r2,0.5076317,0.0190330,0.5086095,0.4854917,0.5339432,0.5162891,0.4938247
residual_deviance,1328.3203,30.498772,1308.66,1370.7126,1317.0199,1348.5276,1296.6813
rmse,0.7187640,0.0102879,0.706745,0.7284689,0.7087284,0.7229697,0.7269078
rmsle,0.3150278,0.0053199,0.3082676,0.3203335,0.3104646,0.3175752,0.3184980


In [41]:
lb = aml.leaderboard

In [42]:
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_2_20221111_151131,0.718685,0.516508,0.543953,0.315009,0.516508
StackedEnsemble_AllModels_3_AutoML_2_20221111_151131,0.719083,0.51708,0.543848,0.31497,0.51708
StackedEnsemble_AllModels_4_AutoML_2_20221111_151131,0.719085,0.517084,0.543797,0.314988,0.517084
StackedEnsemble_BestOfFamily_6_AutoML_2_20221111_151131,0.720213,0.518707,0.546012,0.315583,0.518707
StackedEnsemble_BestOfFamily_4_AutoML_2_20221111_151131,0.720628,0.519304,0.545842,0.315522,0.519304
StackedEnsemble_BestOfFamily_3_AutoML_2_20221111_151131,0.721216,0.520152,0.546625,0.315775,0.520152
StackedEnsemble_AllModels_2_AutoML_2_20221111_151131,0.721221,0.52016,0.54629,0.315831,0.52016
StackedEnsemble_AllModels_5_AutoML_2_20221111_151131,0.721293,0.520264,0.546764,0.315446,0.520264
StackedEnsemble_AllModels_1_AutoML_2_20221111_151131,0.723926,0.524069,0.548376,0.316829,0.524069
StackedEnsemble_BestOfFamily_5_AutoML_2_20221111_151131,0.724256,0.524546,0.548814,0.316382,0.524546


In [43]:
predictions = aml.predict(df_test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


C:\Users\frikk\AppData\Roaming\Python\Python310\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'categorical_chain_name' has levels not trained on: ["24 7 TRENINGSSENTER", "ACE SHOPS", "AVANCIA SPORT", "BADEMILJÃ˜", "BADERINGEN", "BAROKK FRISÃ˜R", "BERTONI NORWAY", "BIRK SPORT", "BLUE ENERGY", "DESINESS", ...23 not listed..., "SALT OG PEPPER HÃ˜NEFOSS", "SKOGSTAD DETALJ", "SKY FITNESS", "TIGER OF SWEDEN", "TIPPY", "VARMEFAG INVEST", "VITA EXCLUSIVE", "YOGIS", "ZAVANNA", "ZAXIZ FRISÃ˜R"]
  warnings.warn(w)
C:\Users\frikk\AppData\Roaming\Python\Python310\site-packages\h2o\job.py:83: UserWarning: Test/Validation dataset column 'categorical_mall_name' has levels not trained on: ["Aamodt Senter", "Bardufoss Torgsenter", "Bellevuesenteret", "BorgeÃ¥sen Senter", "CC Mart'n", "Combisenteret FiskÃ¥", "Coop Mega HÃ¸nefoss", "Coop Mega Ã˜vre Ã…rdal", "Cuben KjÃ¸pesenter", "De Syv SÃ¸stre kjÃ¸pesenter", ...37 not listed..., "StrÃ¸get", "Sundt Motehus", "Svelvik NÃ¦rsenter"

In [44]:
predictions_pd = h2o.as_list(predictions)
predictions_list = predictions_pd["predict"].tolist()
predictions_list = np.expm1(predictions_list)

In [45]:
stores_test = pd.read_csv("../data/stores_test.csv")
stores_test_id = stores_test['store_id'].values

In [46]:
predictions_df = pd.DataFrame()

predictions_df['id'] = stores_test_id
print(len(predictions_list))
predictions_df['predicted'] = predictions_list
predictions_df.to_csv("../predictions/h2o_automl3.csv", index=False)

8577


In [47]:
aml.varimp()

,GLM_1,DeepLearning_1,DeepLearning_grid_3_model_1,DeepLearning_grid_2_model_1,DeepLearning_grid_1_model_1,DeepLearning_grid_2_model_2,DeepLearning_grid_1_model_2,GBM_grid_1_model_18,GBM_grid_1_model_1,GBM_grid_1_model_14,...,GBM_grid_1_model_25,GBM_5,GBM_grid_1_model_13,GBM_grid_1_model_23,GBM_grid_1_model_4,GBM_grid_1_model_27,GBM_grid_1_model_24,GBM_2,GBM_grid_1_model_26,GBM_grid_1_model_9
categorical_street,0.637526,0.758855,0.812029,0.812637,0.785821,0.790171,0.784361,0.083574,0.121757,0.030637,...,0.042626,0.012966,0.085491,0.053595,0.087565,0.033140,0.005132,0.015816,0.035903,0.007176
categorical_chain_name,0.148795,0.078932,0.058393,0.058183,0.071859,0.061767,0.063858,0.133054,0.272781,0.212988,...,0.571935,0.545524,0.376171,0.406864,0.438153,0.420368,0.440502,0.448080,0.357880,0.450878
categorical_sales_channel_name,0.055711,0.030891,0.017795,0.017614,0.022813,0.025647,0.027791,0.086149,0.229668,0.231015,...,0.135099,0.148204,0.126761,0.161367,0.132429,0.231059,0.209699,0.189056,0.260348,0.281572
categorical_mall_name,0.098516,0.095434,0.091359,0.091384,0.092334,0.089690,0.089500,0.051154,0.062079,0.089814,...,0.107497,0.109792,0.061429,0.090502,0.050672,0.088136,0.112806,0.106933,0.095338,0.093841
average_income_per_grunnkrets,0.000125,0.000266,0.000187,0.000183,0.000277,0.000528,0.000389,0.029153,0.018399,0.004364,...,0.010078,0.006271,0.027962,0.015638,0.020175,0.008311,0.002500,0.007545,0.009649,0.004780
feature,0.000223,0.000353,0.000200,0.000197,0.000306,0.000450,0.000560,0.016355,0.009228,0.004723,...,0.006265,0.006599,0.017363,0.009490,0.009665,0.006406,0.004956,0.007212,0.009880,0.006252
features,0.000026,0.000222,0.000192,0.000174,0.000346,0.000439,0.000421,0.014145,0.006038,0.001643,...,0.002990,0.001644,0.009253,0.005805,0.004691,0.001830,0.000656,0.001926,0.003686,0.001634
has_mall_or_not,0.000156,0.000207,0.000200,0.000183,0.000287,0.000408,0.000666,0.002807,0.002187,0.003799,...,0.000166,0.002156,0.002850,0.003333,0.005718,0.002088,0.005225,0.003312,0.005281,0.003097
distance_store_busstop,0.000155,0.000337,0.000252,0.000215,0.000450,0.000800,0.000552,0.036575,0.009953,0.004059,...,0.002982,0.001864,0.023582,0.009566,0.007397,0.003013,0.006231,0.001921,0.003754,0.002351
person_number,0.001127,0.000378,0.000189,0.000180,0.000177,0.000383,0.000338,0.024568,0.018003,0.002042,...,0.004918,0.001195,0.018845,0.008296,0.006296,0.002591,0.001015,0.002249,0.004688,0.001744
